In [2]:
import tweepy as tw
import os
import json
import random
import networkx as nx
from networkx.algorithms.approximation import clique as clq
from pyvis.network import Network

In [3]:
#Twitter API credentials
api_key = "FzUscQQNsdWhkxebVGFZyuHsh"
api_secret = "Tr1OqulWQIYYI6aEpWVOb7U5NL9DoRtCAviXh4tYg4LXg9sBZe"
access_token = "1040603971637923845-9uZqYh42gsCmwoDCB9nTcoSHrHwiV8"
access_secret = "OsZL0PZC9WDw6Xwcl2ASpIisyfkKW3Bwu4BomNgWxAcj0"
bearer_token = "AAAAAAAAAAAAAAAAAAAAAJQUJQEAAAAA%2B5CTzK8KotMS%2FuB61Dlul7ijBE4%3D5uWZb7w1FPD6nb4SpvmnZ4hxGjqGwzfMquWPHQQooS3jkgNg67"

auth = tw.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_secret)
api = tw.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True, timeout=120)
#attributo timeout indica la durata della connessione

if api.verify_credentials:
    print ('Authentication completed successfully!')

Authentication completed successfully!


In [4]:
data_folder = "data"

def serialize_json(folder, filename, data):
    if not os.path.exists(folder):
        os.makedirs(folder, exist_ok=True)
    with open(f"{folder}/{filename}", 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
        f.close()
    print(f"Data serialized to path: {folder}/{filename}")

def read_json(path):
    if os.path.exists(path):
        with open(path, "r", encoding="utf8") as file:
            data = json.load(file)
        print(f"Data read from path: {path}")
        return data
    else:
        print(f"No data found at path: {path}")
        return {}

1. Scaricamento utenti followers ( api.followers ) e utenti following (api.friends ) dei seguenti
    cinque account (i dati numerici potrebbero subire minime variazioni):

    ○ @mizzaro <br>
    ○ @damiano10 <br>
    ○ @Miccighel_ <br>
    ○ @eglu81 <br>
    ○ @KevinRoitero


In [5]:
accounts = ["mizzaro" , "damiano10", "Miccighel_", "eglu81", "KevinRoitero"]
followers = [157, 786, 332, 539, 103]
followings = [331, 837, 212, 621, 256]

accounts --> vettore contenenti gli screen_name dei quali si vogliono scaricare i dati (vettore)<br>
what --> specifica il cosa voglio scaricare (stringa) <br>
quantity --> quantità di followers/followings si vogliono scaricare (intero) <br>
operation --> funzione dell'API da chiamere (funzione)

In [6]:
#download passando vettore di quantità
def download(users, what, quantity, operation):
    i = 0

    for user in users:
        data= []
        print(f"Downloading {what} di {user}")

        for item in tw.Cursor(
            operation,
            screen_name=user,
            skip_status=True,
            include_user_entities=False
        ).items(quantity[i]):

            retriveData(item, data)

        serialize_json(data_folder, f"{user}_{what}s.json", data)
        print(f"ho serializzato un totale di {len(data)} dati \n")
        i += 1

def retriveData(item, data):
    json_data = item._json
    found_follower = {}

    found_follower["id"] = json_data["id"]
    found_follower["name"] = json_data["name"]
    found_follower["screen_name"] = json_data["screen_name"]
    found_follower["location"] = json_data["location"]

    data.append(found_follower)

## Download follower

In [ ]:
download(accounts, "follower", followers, api.followers)
print("Download terminato")

## Download following

In [ ]:
download(accounts, "following", followings, api.friends)
print("Download terminato")

2. Scegliete 5 utenti followers a caso tra quelli di ciascuno dei cinque account e
    scaricate ulteriori 10 utenti followers ( followers dei followers )


In [7]:
def rndIndex(quantity, max):
    rndNum = []

    while len(rndNum) < quantity:
        pos = random.randint(0, max)

        if pos not in rndNum:
            rndNum.append(pos)
    return rndNum

#recupera gli screen_name dei 5 account random
def retriveAccounts(rnd, file):
    data = read_json(f"{data_folder}/{file}")
    newAccounts = []

    for index in rnd:
        #controllo che aaccount selezionato abbia più di 10 followers
        item = data[index]

        if len(api.followers_ids(item["screen_name"])) >= 10 :
            newAccounts.append(item["screen_name"])
            #non gestisce il caso in cui ci siano meno di 10 amici

    return newAccounts

#download con quantity valore intero
def specificQuantityDownload(users, what, quantity, operation):

   for user in users:
        data= []
        print(f"Downloading {what} di {user}")

        for item in tw.Cursor(
            operation,
            screen_name=user,
            skip_status=True,
            include_user_entities=False
        ).items(quantity):

            retriveData(item, data)

        serialize_json(data_folder, f"{user}_{what}s.json", data)
        print(f"ho serializzato un totale di {len(data)} dati \n")

## Download followers of followers

In [ ]:
i = 0
for account in accounts:
    max = followers[i]

    rndNum = rndIndex(5, max)

    specificQuantityDownload(retriveAccounts(rndNum, f"{account}_followers.json"),
                                                f"{account}_followersOfFollower", 10, api.followers)
    i += 1

print("Download terminato")

3. Scegliete 5 utenti following a caso tra quelli di ciascuno dei cinque account e
    scaricate ulteriori 10 utenti following ( following dei following )

## Download followings of followings

In [ ]:
i = 0
for account in accounts:
    max = followings[i]

    rndNum = rndIndex(5, max)

    specificQuantityDownload(retriveAccounts(rndNum, f"{account}_followings.json"),
                                                f"{account}_followingsOfFollowing", 10, api.friends)
    i += 1

print("Download terminato")

4. Scaricare i dettagli del profilo di tutti gli utenti recuperati ( api.get_user )


In [18]:
#si può togliere operation e vedere doc api.get_user
def downloadUserDataFromString(users, output):
    for user in users:
        print(f"Downloading userData di {user}")

        foundUser = api.get_user(user)
        retriveUserData(foundUser, output)

    return output

def downloadUserDataFromObject(users, output):
    for user in users:
        print(f"Downloading userData di {user['screen_name']}")
        try:
            foundUser = api.get_user(user["screen_name"])
            retriveUserData(foundUser, output)

        except: #usato per gestire il caso in cui l'utente abbia il profilo privato
            print("Qualcosa è andato storto ")

    return output

def retriveUserData(item, data):
    json_data = item._json

    profile_data = {}

    profile_data["id"] = json_data["id"]
    profile_data["name"] = json_data["name"]
    profile_data["screen_name"] = json_data["screen_name"]
    profile_data["location"] = json_data["location"]
    profile_data["description"] = json_data["description"]
    profile_data["followers_count"] = json_data["followers_count"]

    if profile_data not in data:
        data.append(profile_data)


## Download info utenti

In [20]:
lstFile = os.listdir(data_folder)#lista dei file contenuti nella directory "data"

if "usersData.json" not in lstFile:

    userData = []
    #download user data degli uterti di partenza(mizzaro, damiano10,...)
    downloadUserDataFromString(accounts, userData)

    for file in lstFile:
       jsonItems = read_json(f"{data_folder}/{file}")

       downloadUserDataFromObject(jsonItems, userData)

    print(f"ho serializzato un totale di {len(userData)} dati \n")

    serialize_json(data_folder, f"usersData.json", userData)
else:
    print("I dati utente sono già stati serializzati in precedenza")

print("Download terminato")

#PROBLEMA: ad ogni esecuzione cambia il numero di elementi scaricati


Data read from path: data/ABCFactCheck_damiano10_followingsOfFollowings.json
Data read from path: data/ABH878_mizzaro_followersOfFollowers.json
Data read from path: data/ACMSIGIR_KevinRoitero_followingsOfFollowings.json
Data read from path: data/airamoigroig_mizzaro_followingsOfFollowings.json
Data read from path: data/arjenpdevries_KevinRoitero_followersOfFollowers.json
Data read from path: data/BrachaShapira_damiano10_followingsOfFollowings.json
Data read from path: data/CaRRws_mizzaro_followingsOfFollowings.json
Data read from path: data/CreatoursUK_Miccighel__followersOfFollowers.json
Data read from path: data/CriMuntean_KevinRoitero_followersOfFollowers.json
Data read from path: data/cs2_school_KevinRoitero_followersOfFollowers.json
Data read from path: data/CSSamsterdam_damiano10_followersOfFollowers.json
Data read from path: data/dailyrando_Miccighel__followersOfFollowers.json
Data read from path: data/DameWendyDBE_eglu81_followingsOfFollowings.json
Data read from path: data/dam

Rate limit reached. Sleeping for: 286
Rate limit reached. Sleeping for: 481
Rate limit reached. Sleeping for: 480
Rate limit reached. Sleeping for: 448
Rate limit reached. Sleeping for: 423


5. Costruite la rete sociale (grafo):

    ○ Inserite l’ id di ciascun utente come identificatore del nodo <br>
    ○ Ogni arco rappresenta una relazione follows tra due utenti <br>
    ○ La presenza della relazione va verificata tra tutti gli utenti scaricati ed i cinque account iniziali ( api.show_friendship ) <br>
    ○ Inserite i dettagli del profilo di ciascun utente come attributi del nodo <br>
    ○ Inserite i membri del vostro gruppo come attributi del grafo <br>
    ○ Per ogni nodo, aggiungete un attributo con il numero di follower individuati


In [37]:
nodes = read_json(f"{data_folder}/usersData.json")
twtNet = nx.Graph()

print(nx.number_of_nodes(twtNet))

Data read from path: data/usersData.json
0


6. Producete una visualizzazione interattiva del grafo usando pyvis


In [ ]:
#modificare parametri per cambiare la parte grafica
net = Network(
    height = "100%",
    width = "100%",
    bgcolor = "#222222",
    font_color = "white",
    heading = "Rete Sociale"
)

net.barnes_hut()#si possono cambiare parametri
net.from_nx(twtNet)
neighbor_map = net.get_adj_list()

for node in net.nodes:
    node["value"] = len(neighbor_map[node["id"]])

net.show("twtNet.html")

7. Verificate se il grafo:

    ○ è connesso ( is_connected ) <br>
    ○ è bipartito ( is_bipartite )


In [ ]:
if  nx.is_connected(twtNet):
    print("Il grafo è connesso")
else:
    print("Il grafo non è connesso")

if nx.is_bipartite(twtNet):
    print("Il grafo è bipartito")
else:
    print("Il grafo non è bipartito")

8. Misurate le seguenti distanze sul grafo:

    ○ Centro ( center ) <br>
    ○ Diametro ( diameter ) <br>
    ○ Raggio ( radius )


In [ ]:
print(f"Centro: {nx.center(twtNet)}")
print(f"Diametro: {nx.diameter(twtNet)}")
print(f"Raggio: {nx.radius(twtNet)}") #forse serve qualche parametro

9. Calcolate le seguenti misure di centralità sul grafo:

    ○ Betweenness centrality ( betweenness_centrality ) <br>
    ○ Closeness centrality ( closeness_centrality ) <br>
    ○ Degree centrality ( degree_centrality ) <br>
    ○ In-degree centrality ( in_degree_centrality ) <br>
    ○ Out-degree centrality ( out_degree_centrality ) <br>
    ○ Page Rank ( pagerank ) <br>
    ○ HITS ( hits )


In [ ]:
betweennessCentrality = nx.betweenness_centrality(twtNet)
closenessCentrality = nx.closeness_centrality(twtNet)
degreeCentrality =  nx.degree_centrality(twtNet)
inCentrality = nx.in_degree_centrality(twtNet)
outCentrality = nx.out_degree_centrality(twtNet)

print(f"Betweenness centrality: {betweennessCentrality}") #forse serve qualche parametro
print(f"Closeness centrality: {closenessCentrality}") #forse serve qualche parametro
print(f"Degree centrality: {degreeCentrality}") #forse serve qualche parametro
print(f"IN degree centrality: {inCentrality}") #forse serve qualche parametro
print(f"OUT degree centrality: {outCentrality}") #forse serve qualche parametro
print(f"Page rank: {nx.pagerank(twtNet)}") #forse serve qualche parametro
print(f"HITS: {nx.hits(twtNet)}") #forse serve qualche parametro


10. Generate un sottografo del grafo ( ego_graph ) e calcolate::

    ○ Cricca massima ( max_clique ) <br>
    ○ Dimensione della cricca massima ( large_clique_size )

Si è deciso di calcolare la cricca per Kevin Roitero per motivi di complessità

In [ ]:
roiteroGraph = nx.ego_graph(twtNet, "Brynden-Tully", radius=1)

clique = clq.max_clique(roiteroGraph)
cliquegraph = twtNet.subgraph(clique)

dimensione = nx.number_of_nodes(cliquegraph) + nx.number_of_edges(cliquegraph)
print(f"La dimensione del sottografo calcolato sulla cricca massima vale: {dimensione}")

11. Calcolate la copertura minima degli archi ( min_edge_cover ) del grafo


In [ ]:
minCover = nx.min_edge_cover(twtNet)
print(f"Copertura minima arco: {minCover}")


12. Calcolate i seguenti coefficienti per stimare la “small-world-ness” del grafo:

    ○ Coefficiente omega ( omega ) <br>
    ○ Coefficiente sigma ( sigma )

In [ ]:
omega = nx.omega(twtNet) #forse serve qualche parametro
sigma = nx.sigma(twtNet) #forse serve qualche parametro

print(f"Coefficiente omega: {omega}")
print(f"Coefficiente sigma: {sigma}")


13. Calcolare la correlazione di Pearson Rho e di Kendall Tau fra le misure di centralità;
    riportare il risultato in due tabelle

In [ ]:
rho = nx.degree_pearson_correlation_coefficient(betweennessCentrality, closenessCentrality, degreeCentrality, inCentrality, outCentrality) #forse serve qualche parametro e controllare la funzione
tau = 0 #da calcolare

print(f"Coefficiente di Pearso: {rho}")
print(f"Coefficiente di Kendal: {tau}")
